In [3]:
import neilpy
import pandas as pd
import glob
import os

# Load image names into a list
images_dir = 'POAS/*.jpg'
fns = glob.glob(images_dir)

# Read the geotags into a dataframe
photos_df = neilpy.read_geotags_into_df(fns,return_datetimes=False)

# Fix names, as we don't want to include the path to the image, just the basename
photos_df['fn'] = photos_df['fn'].apply(os.path.basename)

# Calculate the azimuths based on the tracks
photos_df['azimuth'] = neilpy.track2azimuth(photos_df['lat'].values,photos_df['lon'].values)

# Based on a specified pitch estimated during the mounting, calculate Omega, Phi, and Kappa angles
pitch = -70
photos_df['omega'],photos_df['phi'],photos_df['kappa'] = neilpy.ypr2opk(photos_df['azimuth'].values,pitch)

# Correct for GEOID
photos_df['alt'] = photos_df['alt'] + 35.356 

# Define accuracy of measurements:
photos_df['xy_acc'] = 2.
photos_df['z_acc'] = 2.

# Write out the values
outfile = 'sept_poas_opk.csv'
cols = ['fn','lat','lon','alt','omega','phi','kappa','xy_acc','z_acc']
photos_df.to_csv(outfile,index=False,header=False,columns=cols)